In [1]:
%load_ext lab_black

# EDA

## Author: Hunter Merrill

This notebook is scratch space for summaries, statistics, and visualizations that will help make modeling decisions. I often refered to the data guide while writing this notebook: https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Dataset_Documentation/DVS/natality/UserGuide2018-508.pdf

In [2]:
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_theme(style="ticks")

from constants import MISSING_CODE, VARIABLE_TYPE

ModuleNotFoundError: No module named 'constants'

In [ ]:
data = pd.read_csv("../../data/train.csv")

# Sanity checks

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
len(data["id"].unique()) == data["id"].shape[0]

In [ ]:
data.describe()

The `id` variable is indeed a unique identifier. No missing values in the response variable `DBWT`. Four explicitly categorical features:

In [ ]:
data["DMAR"].unique()  # marital status

In [ ]:
data["LD_INDL"].unique()  # induction of labor

In [ ]:
data["RF_CESAR"].unique()  # previous Cesarean

In [ ]:
data["SEX"].unique()  # sex of infant

No `NaN`s in the features but there are missing values, see `constants.py` for details.

# Numerical features

In [ ]:
features = VARIABLE_TYPE.keys()

for feature in features:
    if "categorical" not in VARIABLE_TYPE[feature]:
        missing = (
            MISSING_CODE[feature]
            if isinstance(MISSING_CODE[feature], list)
            else [MISSING_CODE[feature]]
        )
        data.query(f"~{feature}.isin({missing})")[feature].hist(bins=100)
        plt.title(feature)
        plt.show()

`CIG_0`, `PRIORDEAD`, `PRIORTERM`, and `RF_CESARN` are skewed enough to be a potential problem. See if unskewing helps:


In [ ]:
for feature in ["CIG_0", "PRIORDEAD", "PRIORTERM", "RF_CESARN"]:
    df_plot = data.query(f"{feature} != {MISSING_CODE[feature]}")
    plt.scatter(df_plot[feature], df_plot["DBWT"], alpha=0.01)
    plt.title(feature)
    plt.show()
    plt.scatter(np.log(df_plot[feature] + 1), df_plot["DBWT"], alpha=0.01)
    plt.title("log" + feature)
    plt.show()

# Pair plots

In [ ]:
data_for_pairplot = data.copy()

# convert that timestamp column to contiguous numbers
fake_datetime = "1970-01-01 " + data_for_pairplot["DOB_TT"].astype(str).str.zfill(4)
fake_datetime = fake_datetime.apply(lambda x: None if "9999" in x else x)
data_for_pairplot["DOB_TT"] = (
    pd.to_datetime(fake_datetime, format="%Y-%m-%d %H%M")
    - datetime.datetime(1970, 1, 1)
).dt.total_seconds()

# convert implicit to explicit categorical columns
for feature in VARIABLE_TYPE.keys():
    if "categorical" in VARIABLE_TYPE[feature]:
        data_for_pairplot[feature] = data_for_pairplot[feature].astype(str)
    else:
        data_for_pairplot[feature] = data_for_pairplot[feature].astype(float)

# fill missing with NAs
for feature in MISSING_CODE.keys():
    missing = (
        MISSING_CODE[feature]
        if isinstance(MISSING_CODE[feature], list)
        else [MISSING_CODE[feature]]
    )
    data_for_pairplot.loc[data_for_pairplot[feature].isin(missing), feature] = None

# drop unwanted columns
data_for_pairplot = data_for_pairplot.drop(["PAY", "ILP_R"], axis=1)

# binarize that one column
data_for_pairplot["ILLB_R"] = (data_for_pairplot["ILLB_R"] <= 3).astype(str)

sns.pairplot(
    data_for_pairplot.select_dtypes(include="number").drop("id", axis=1),
    kind="scatter",
    plot_kws={"alpha": 0.1},
)
plt.show()

# Categorical effects

In [ ]:
data_for_violinplots = data_for_pairplot.assign(
    id=lambda x: x["id"].astype(str)
).select_dtypes(include="object")
data_for_violinplots = data_for_violinplots.merge(
    data[["id", "DBWT"]].assign(id=lambda x: x["id"].astype(str)),
    on="id",
    how="inner",
    validate="1:1",
)
data_for_violinplots = data_for_violinplots.fillna("None")

for feature in set(VARIABLE_TYPE.keys()) - {"PAY"}:
    if "categorical" in VARIABLE_TYPE[feature]:
        plt.violinplot(
            [
                data_for_violinplots.query(f"{feature} == @x")["DBWT"]
                for x in data_for_violinplots[feature].unique()
            ]
        )
        plt.title(feature)
        plt.show()

# Test set

In [ ]:
data_test = pd.read_csv("../data/test.csv")

for feature in features:
    if "categorical" in VARIABLE_TYPE[feature]:
        unique_values_train = set(data[feature].unique())
        unique_values_test = set(data_test[feature].unique())
        if unique_values_train.union(unique_values_test) != unique_values_train:
            print(f"categorical feature {feature} takes new values in the test set.")
    else:
        min_train, max_train = (
            data.query(f"{feature} != {MISSING_CODE[feature]}")[feature].min(),
            data.query(f"{feature} != {MISSING_CODE[feature]}")[feature].max(),
        )
        min_test, max_test = (
            data_test.query(f"{feature} != {MISSING_CODE[feature]}")[feature].min(),
            data_test.query(f"{feature} != {MISSING_CODE[feature]}")[feature].max(),
        )
        if (min_test < min_train) or (max_test > max_train):
            print(f"Possible extrapolation problem for {feature}.")
            print(f"Train range: \t{min_train}, {max_train}")
            print(f"Test range: \t{min_test}, {max_test}")

In [ ]:
data.query(f"M_Ht_In != {MISSING_CODE['M_Ht_In']}")["M_Ht_In"].hist(
    bins=50, density=True, alpha=0.75
)
data_test.query(f"M_Ht_In != {MISSING_CODE['M_Ht_In']}")["M_Ht_In"].hist(
    bins=50, density=True, alpha=0.75
)
plt.show()

In [ ]:
plt.scatter(data["M_Ht_In"], data["DBWT"], alpha=0.01)
plt.show()

# Response variable

In [ ]:
data["DBWT"].hist(bins=100)

# Discoveries of Note

Marginally, the strongest signals seem to come from `BFACIL` (birth place), `NO_INFEC` (whether an infection was reported), `WTGAIN` (weight gain), and `PWgt_R` (pre-pregnancy weight).

As expected, `PWgt_R` and `BMI` are strongly correlated, and it may be worth dropping one (or de-correlating them).

`ILLB_R`, the number of months since last live birth, implicitly takes four "types:"
* values less than 3 mean "pleural delivery"
* values 3-300 are actually months since last live birth
* 888 means this is the first birth, so "months since last live birth" is undefined
* 999 means missing.

However, it might be possible to simply binarize this variable:

In [ ]:
plt.scatter(np.log(data["ILLB_R"] + 1), data["DBWT"], alpha=0.1)
plt.show()

`ILP_R`, the interval since last pregnancy, is obviously related, and it might be wise to simply drop one. The difference would be previous terminations, which is covered in `PRIORTERM`.

In [ ]:
plt.scatter(np.log(data["ILP_R"] + 1), data["DBWT"], alpha=0.1)
plt.show()

In [ ]:
plt.scatter(np.log(data["ILP_R"] + 1), np.log(data["ILLB_R"] + 1), alpha=0.1)
plt.show()

In [ ]:
print(data["ILP_R"].value_counts())
print(data["ILLB_R"].value_counts())

`ILOP_R`seems related but in fact shows far fewer pleural pregnancies (and may clarify a trend for very short intervals), so binarization is not as attractive, although `888` = "not applicable" and `999` = "unknown" can be safely combined:

In [ ]:
plt.scatter(np.log(data["ILOP_R"] + 1), data["DBWT"], alpha=0.1)
plt.show()

In [ ]:
plt.violinplot([data.query(f"ILOP_R == {x}")["DBWT"] for x in [888, 999]])
plt.show()

The two are in fact not very related to each other.

In [ ]:
plt.scatter(np.log(data["ILOP_R"] + 1), np.log(data["ILLB_R"] + 1), alpha=0.1)
plt.show()

`PAY` and `PAY_REC` are very similar (`PAY` categories 4, 5, 6, and 8 are combined into `PAY_REC` category 4). They also have the same missingness. Marginally, neither seem important, so likely we can drop `PAY`:

In [ ]:
plt.violinplot([data.query(f"PAY == {x}")["DBWT"] for x in data["PAY"].unique()])
plt.show()

In [ ]:
plt.violinplot(
    [data.query(f"PAY_REC == {x}")["DBWT"] for x in data["PAY_REC"].unique()]
)
plt.show()

`PRECARE` is the month during which pre-care began (0 if no care, 99 if missing):

In [ ]:
plt.violinplot(
    [data.query(f"PRECARE == {x}")["DBWT"] for x in [0] + list(range(1, 11)) + [99]]
)
plt.show()

No strong signal. Should we combine it with `DOB_MM` to engineer the feature "months before birth that care began"? Seems like no.

In [ ]:
data["precare_months_before_birth"] = (data["DOB_MM"] - data["PRECARE"]).apply(
    lambda x: x % 12
)

plt.violinplot(
    [
        data.query("PRECARE != 99").query(f"precare_months_before_birth == {x}")["DBWT"]
        for x in range(12)
    ]
)
plt.show()

# Decisions

For the simple regression model:
* Log-transform `CIG_0`, `PRIORDEAD`, `PRIORTERM`, and `RF_CESARN`
* Replace `BMI` with the residual from `BMI ~ PWgt_R`
* Impute missing numerical values with a median, categorical values with a mode
* Polynomial effect on `ILOP_R`

For the tree-based quantile regression model:
* Leave all missing coded
* No log-transforms

For the Bayesian neural network:
* Replace missing codes with real missing and use the expected relu layer
* Include missingness indicators as features
* Decorrelate all features
* Log-transform `CIG_0`, `PRIORDEAD`, `PRIORTERM`, and `RF_CESARN`

For all models:
* Binarize `ILLB_R` to `<=3` and `>3`
* Drop `ILP_R` (it has more missing than the almost-equivalent `ILLB_R`)
* Set `888` and `999` to "missing" for `ILOP_R`
* Drop `PAY` (seems that `PAY_REC` covers it)
* Clip `M_Ht_In` to [48, 78]

Two submissions:
* Best of three
* Linear quantile regression ensemble model